https://github.com/facebookresearch/large_concept_model/tree/main

In [1]:
!git clone https://github.com/facebookresearch/large_concept_model.git

Cloning into 'large_concept_model'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 532 (delta 152), reused 127 (delta 127), pack-reused 234 (from 1)
Receiving objects: 100% (532/532), 800.00 KiB | 7.55 MiB/s, done.
Resolving deltas: 100% (244/244), done.


In [1]:
%cd large_concept_model

/content/large_concept_model


In [3]:
!uv sync --extra cpu --extra eval --extra data

Using CPython 3.12.11 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 192 packages in 426ms
Prepared 186 packages in 17m 07s
Installed 187 packages in 1.28s
 + absl-py==2.1.0
 + accelerate==1.2.0
 + adapters==1.0.1
 + aiohappyeyeballs==2.4.4
 + aiohttp==3.11.10
 + aiosignal==1.3.2
 + alabaster==1.0.0
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + attrs==24.2.0
 + babel==2.16.0
 + bert-score==0.3.13
 + blis==0.7.11
 + blobfile==2.1.1
 + botok==0.8.12
 + cached-property==2.0.1
 + catalogue==2.0.10
 + certifi==2024.8.30
 + cffi==1.17.1
 + charset-normalizer==3.4.0
 + click==8.1.7
 + cloudpathlib==0.20.0
 + cloudpickle==3.1.0
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + confection==0.1.5
 + contourpy==1.3.1
 + cycler==0.12.1
 + cymem==2.0.10
 + dacite==1.8.1
 + datasets==3.2.0
 + dill==0.3.8
 + docopt==0.6.2
 + docutils==0.21.2
 + editdistance==0.8.1
 + emoji==2.14.0
 + en-core-web-sm==3.7.1 (from https://github.com/explosion/spacy-models/rel

In [4]:
!uv pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
!pip install -e ".[data,eval]"

Using Python 3.12.11 environment at: /usr
Resolved 14 packages in 737ms
Prepared 2 packages in 411ms
Uninstalled 4 packages in 932ms
Installed 4 packages in 290ms
 - sympy==1.13.3
 + sympy==1.13.1
 - torch==2.8.0+cu126
 + torch==2.5.1+cpu
 - torchaudio==2.8.0+cu126
 + torchaudio==2.5.1+cpu
 - torchvision==0.23.0+cu126
 + torchvision==0.20.1+cpu
Using Python 3.12.11 environment at: /usr
Resolved 42 packages in 1.14s
Prepared 2 packages in 35ms
Uninstalled 5 packages in 82ms
Installed 11 packages in 37ms
 - blobfile==3.0.0
 + blobfile==2.1.1
 + colorama==0.4.6
 + fairseq2==0.3.0rc1
 + fairseq2n==0.3.0rc1+cpu
 - importlib-metadata==8.7.0
 + importlib-metadata==7.2.1
 - lxml==5.4.0
 + lxml==4.9.4
 - numpy==2.0.2
 + numpy==1.26.4
 - packaging==25.0
 + packaging==23.2
 + portalocker==3.2.0
 + sacrebleu==2.5.1
 + torcheval==0.0.7
Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting require

In [ ]:
pip install --upgrade pip
pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
pip install -e ".[data,eval]"

In [3]:
!uv run --extra data prepare_evaluation_data.py prepare_data \
    --dataset_name=cnn_dailymail \
    --output_dir=jsonl_dataset \
    --source_text_column=article \
    --target_text_column=highlights \
    --version=3.0.0 \
    --prompt_prefix="Summarize the following news to a concise list of highlights.\n[Text Start]:\n"\
    --prompt_suffix="\n[Text End]"

error: Failed to spawn: `prepare_evaluation_data.py`
  Caused by: No such file or directory (os error 2)


In [6]:
launcher = Launcher(
      cache=None,
      config_dump_dir=Path(log_dir) / "conf",
      log_folder=Path(log_dir) / "logs",
      cluster=mode,
      update_parameters={"partition": "your_slurm_partition"},
  )
_ = await launcher.schedule(inst_stopes_module)

NameError: name 'Launcher' is not defined

In [9]:
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3  \
  --model_name meta-llama/Llama-3.1-8B-Instruct \
  --generator_batch_size 16 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 200}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 16 \
  --dataset.soure_text_column prompt \
  --dataset.source_target_column answer \
  --dump_dir output_results

usage: __main__.py [-h] [--cfg CFG] [--columns COLUMNS]
                   [--source_text_column SOURCE_TEXT_COLUMN]
                   [--target_text_column TARGET_TEXT_COLUMN]
                   [--source_prefix_text SOURCE_PREFIX_TEXT]
                   [--source_suffix_text SOURCE_SUFFIX_TEXT]
                   [--target_prefix_text TARGET_PREFIX_TEXT]
                   [--target_suffix_text TARGET_SUFFIX_TEXT]
                   [--source_sequences SOURCE_SEQUENCES]
                   [--target_sequences TARGET_SEQUENCES]
                   [--silent_freeze SILENT_FREEZE] [--file_path FILE_PATH]
                   [--prompt_template PROMPT_TEMPLATE]
__main__.py: error: unrecognized arguments: --soure_text_column prompt --source_target_column answer
E0822 22:17:07.322000 7457 torch/distributed/elastic/multiprocessing/api.py:869] failed (exitcode: 2) local_rank: 0 (pid: 7469) of binary: /content/large_concept_model/.venv/bin/python
Traceback (most recent call last):
  File "/cont

In [8]:
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3  \
  --model_name meta-llama/Llama-3.2-1B-Instruct \
  --generator_batch_size 16 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 200}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 16 \
  --dataset.soure_text_column prompt \
  --dataset.source_target_column answer \
  --dump_dir output_results

/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:39: SyntaxWarning: invalid escape sequence '\s'
  ('\s\s+', ' '),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:40: SyntaxWarning: invalid escape sequence '\d'
  ('ln\d ', ''),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:41: SyntaxWarning: invalid escape sequence '\g'
  ('([^༅།་ ]) །', '\g<1>་ །'),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:43: SyntaxWarning: invalid escape sequence '\d'
  ('([^་།\d] )', '\g<1>-'),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:43: SyntaxWarning: invalid escape sequence '\g'
  ('([^་།\d] )', '\g<1>-'),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:44: SyntaxWarning: invalid

In [10]:
%cd examples/evaluation

/content/large_concept_model/examples/evaluation


In [12]:
!python /content/large_concept_model/examples/evaluation/prepare_evaluation_data.py --config c/content/large_concept_model/examples/evaluation/instruction.yaml

Traceback (most recent call last):
  File "/content/large_concept_model/examples/evaluation/prepare_evaluation_data.py", line 28, in <module>
    from stopes.modules.preprocess.sonar_text_embedding import (
  File "/usr/local/lib/python3.12/dist-packages/stopes/modules/preprocess/sonar_text_embedding.py", line 17, in <module>
    from fairseq2.assets.error import AssetError
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chat

In [13]:
%cd /content/large_concept_model

/content/large_concept_model


In [14]:
!pip install -e ".[data,eval]"

Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.0 MB/s eta 0:00:00
  Building editable for large-concept-model (pyproject.toml) ... done
  Created wheel for large-concept-model: filename=large_concept_model-0.1.0-py3-none-any.whl size=7805 sha256=da68c65e70e9041549465b9169c8fe4ff2c64a782bf45b483c72603ecf45aeb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-lq3k2oa4/wheels/e0/87/32/9c89b474bef734af9b36cb13df39a74c4f0c331eb2b1a29025
Successfully built large-concept-model
  Attempting uninstall: large-concept-model
    Found existing installation: large-concept-model 0.1.0
    Uninstalling large-concept-model-0.1.0:
      Successfully uninstalled large-concept-model-0.1.0


In [16]:
%cd /content/large_concept_model/lcm
!uv run /content/large_concept_model/lcm/datasets/base.py

/content/large_concept_model/lcm


In [17]:
%cd /content/large_concept_model/lcm/evaluation
!uv run /content/large_concept_model/lcm/evaluation/run.py

/content/large_concept_model/lcm/evaluation


In [18]:
%cd /content/large_concept_model/lcm/inference/lcm
!uv run /content/large_concept_model/lcm/inference/lcm/generator.py

/content/large_concept_model/lcm/inference/lcm
/content/large_concept_model/lcm/nn/schedulers/ddim.py:81: SyntaxWarning: invalid escape sequence '\p'
  math:: $\bar alpha(t) = {cos((t/T + s) / (1+s) * \pi/2)}^2$, HF:diffusers sets `s` to 0.008.


In [19]:
!!uv run generator.py --prompt "dog" --output_dir "/content"

[]

In [20]:
%cd /content/large_concept_model/examples/evaluation
!uv run /content/large_concept_model/examples/evaluation/prepare_evaluation_data.py

/content/large_concept_model/examples/evaluation
/content/large_concept_model/.venv/lib/python3.12/site-packages/docopt.py:165: SyntaxWarning: invalid escape sequence '\S'
  name = re.findall('(<\S*?>)', source)[0]
/content/large_concept_model/.venv/lib/python3.12/site-packages/docopt.py:166: SyntaxWarning: invalid escape sequence '\['
  value = re.findall('\[default: (.*)\]', source, flags=re.I)
/content/large_concept_model/.venv/lib/python3.12/site-packages/docopt.py:207: SyntaxWarning: invalid escape sequence '\['
  matched = re.findall('\[default: (.*)\]', description, flags=re.I)
/content/large_concept_model/.venv/lib/python3.12/site-packages/docopt.py:456: SyntaxWarning: invalid escape sequence '\S'
  split = re.split('\n *(<\S+?>|-\S+?)', doc)[1:]
NAME
    prepare_evaluation_data.py

SYNOPSIS
    prepare_evaluation_data.py GROUP | COMMAND | VALUE

GROUPS
    GROUP is one of the following:

     json
       JSON (JavaScript Object Notation) <https://json.org> is a subset of JavaS

In [23]:
%cd /content/large_concept_model/lcm/inference/lcm
!uv run /content/large_concept_model/lcm/inference/lcm/generator.py -h

/content/large_concept_model/lcm/inference/lcm


In [24]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
#

from dataclasses import dataclass
from typing import List, Optional, Tuple

import torch
from fairseq2.generation.generator import (
    GenerationCounters,
    Hypothesis,
    SequenceGeneratorOutput,
)
from fairseq2.logging import get_log_writer

from lcm.datasets.batch import EmbeddingsBatch, PaddingMask
from lcm.models.abstract_lcm import AbstractLCModel
from lcm.nn.incremental_state import LCMIncrementalStateBag

logger = get_log_writer(__name__)


"""
This generator follows the style of existing generators in Fairseq2
"""


@dataclass
class LCMGeneratorOptions:
    """Holds the options to pass to a sequence generator."""

    max_seq_len: int = 200
    """The hard limit on maximum length of generated sequences."""

    min_seq_len: int = 1
    """The minimum length of generated sequences."""

    eos_threshold: Optional[float] = 0.9
    """Threshold for cosine similarity to the EOS vector"""

    sample_latent_variable: bool = True
    """When using VAE models, whether to return the mean or sample"""

    stop_on_repetition_cosine_threshold: Optional[float] = None
    """Stop the generation when the similarity of two consecutive concepts is above the threshold."""

    include_eos_token: bool = False
    """Whether the eos token should be included in the hypotheses (matters only if they are trimmed)."""

    trim_hypotheses: bool = False
    """Whether the tokens after the EOS token should be included in the hypotheses."""

    seed: Optional[int] = None
    """Seed to make generation deterministic"""

    lcm_temperature: float = 1.0
    """Temperature for decoding in the LCM"""


class LCMGenerator:
    """Generates with an LCM model."""

    def __init__(
        self,
        model: AbstractLCModel,
        options: Optional[LCMGeneratorOptions] = None,
        eos_vec: Optional[torch.Tensor] = None,
    ) -> None:
        """
        :param model:
            The LC model to use for generation.
        """
        model.eval()
        self.model = model

        if options is None:
            options = LCMGeneratorOptions()

        self.eos_vec = eos_vec
        if self.eos_vec is None and options.eos_threshold:
            logger.warning(
                f"eos_threshold is set to {options.eos_threshold}, but eos_vec is not provided"
            )
        if options.eos_threshold:
            logger.debug(f"The eos_vec in generator has been set to {self.eos_vec}")

        self.options = options

        self.max_seq_len = options.max_seq_len
        self.min_seq_len = options.min_seq_len

        assert self.min_seq_len >= 1, (
            f"min_seq_len must be greater than or equal to 1, min_seq_len={options.min_seq_len}"
        )

        self.eos_threshold = options.eos_threshold

        self.seqs: torch.Tensor
        self.step_nr = 0
        self.min_prompt_len: int
        self.max_prompt_len: int
        self.sample_indices: torch.Tensor
        self.state_bag: Optional[LCMIncrementalStateBag] = None
        self.prompt_seq_lens: Optional[torch.Tensor] = None
        self.prompt_padding_mask: Optional[torch.Tensor] = None
        self.lengths: torch.Tensor
        self.step_scores: torch.Tensor

    @torch.inference_mode()
    def __call__(
        self,
        batch_input: EmbeddingsBatch,
        max_gen_len: Optional[int] = None,
        min_gen_len: Optional[int] = None,
        temperature: float = 0.0,
        disable_cache: bool = False,
        **kwargs,
    ) -> SequenceGeneratorOutput:
        """
        :param input:
            `bacth_input` embedded and padded tensor sequence of the inputs
            `max_gen_len` max length to be generated for the given input
            `min_gen_len` minimum length to be generated for the given input
            `temperature` temperature to control the generation
            `disable_cache` if True, do not use kv-caching
        :returns:
            The output of the LCM generator, consists of :math:`N` lists of
            hypotheses for :math:`N` prompts. Each list has 1 Hypothesis
            (beam size = 1), of which `seq` has the  *Shape:* math:`(S+T, D)`
            (:math:`S` is the prompt length, :math:`T` the length of the
            generated sequence after the prompt and :math:`D` the model
            dimension.)

        """
        if self.options.seed:
            torch.manual_seed(self.options.seed)

        # Setup the variables
        batch_size, self.max_prompt_len, embed_dim = batch_input.seqs.size()
        prompt_padding_mask = batch_input.padding_mask
        if prompt_padding_mask is None:
            self.min_prompt_len = self.max_prompt_len
            self.prompt_padding_mask = None
            self.prompt_seq_lens = None
        else:
            self.prompt_seq_lens = prompt_padding_mask.seq_lens
            assert self.prompt_seq_lens is not None, (
                "Expecting a valid `self.prompt_seq_lens` Tensor, found `None`"
            )
            self.min_prompt_len = int(torch.min(self.prompt_seq_lens, dim=0)[0].item())

            # Keep the materialized mask
            self.prompt_padding_mask = prompt_padding_mask.materialize()

        if not max_gen_len:
            max_gen_len = self.max_seq_len

        # Make sure we do not accidentally set a max_gen_len that exceeds
        # the generator's model capability
        assert max_gen_len <= self.max_seq_len, (
            f"Generator can generate up to {self.max_seq_len} sequences, max_gen_len={max_gen_len}"
        )
        self.max_gen_len = max_gen_len

        if not min_gen_len:
            min_gen_len = self.min_seq_len

        assert min_gen_len > 0, (
            f"min_gen_len must be greater than or equal to 1, min_gen_len={min_gen_len}"
        )
        self.min_gen_len = min_gen_len

        if temperature == 0.0:
            # If the call doesn't pass a specific temperature,
            # use the default one from the decoding options
            temperature = self.options.lcm_temperature

        self.temperature = temperature

        for k, v in kwargs.items():
            if hasattr(self.options, k) and v:
                setattr(self.options, k, v)

        # Holds the generated sequences, scores and sample-dependent variables
        dtype = self.model.dtype
        device = batch_input.seqs.device

        if disable_cache:
            self.state_bag = None
        else:
            self.state_bag = LCMIncrementalStateBag(
                self.max_prompt_len + self.max_gen_len
            )

        # reserving full sequences capacity
        self.seqs = torch.zeros(
            (batch_size, self.max_prompt_len + self.max_gen_len, embed_dim),
            device=device,
            dtype=dtype,
        )
        self.step_scores = torch.zeros(
            (batch_size, self.max_prompt_len + self.max_gen_len),
            device=device,
        )
        self.lengths = torch.zeros(batch_size, dtype=torch.int, device=device) - 1

        # Hold the samples indices to return in order
        self.sample_indices = torch.arange(batch_size, device=device)
        # Output buffer
        self.hypotheses: List[List[Hypothesis]] = [[] for _ in range(batch_size)]

        # Bootstrap the sequences with the provided prompt.
        self.seqs[:, : self.max_prompt_len] = batch_input.seqs[:, : self.max_prompt_len]
        self.step_nr = self.min_prompt_len
        self.prefill(**kwargs)

        for self.step_nr in range(
            self.min_prompt_len, self.max_prompt_len + self.max_gen_len
        ):
            if not self._step():
                break

        return SequenceGeneratorOutput(self.hypotheses, counters=GenerationCounters())

    @torch.inference_mode()
    def prefill(self, **kwargs) -> None:
        """The initial forward pass in the decoder with the prefix/prompt
        to populate the KV-cache"""

        if self.state_bag is None:
            return

        # Prefilling with -1 since the next call to step will use the last token in the prefix
        prefill_len = self.step_nr - 1

        if prefill_len > 0:
            _ = self._decode(
                self.seqs[:, :prefill_len],
                padding_mask=None,
            )
            self.state_bag.increment_step_nr(prefill_len)  # type: ignore
        else:
            logger.warning(
                f"Skipping prefill since only a context size of {self.step_nr} is provided in the prefix"
            )

    @torch.inference_mode()
    def _decode(
        self,
        seqs: torch.Tensor,
        padding_mask: Optional[PaddingMask],
        **kwargs,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        output = self.model.predict_next_sentence(
            EmbeddingsBatch(seqs, padding_mask),
            sample=self.options.sample_latent_variable,
            temperature=self.temperature,
            state_bag=self.state_bag,
            **kwargs,
        )

        # Dummy scores
        scores = torch.zeros(seqs.shape[:-1])
        return output.seqs, scores

    def _step(self) -> bool:
        # Generate the next step output.

        if self.state_bag is None:
            # Without a state_bag, we're forwarding the full prefix
            model_output, step_score = self._decode(
                seqs=self.seqs[:, : self.step_nr],
                padding_mask=None,
            )
        else:
            # Since we're using a state_bag, we're only forwarding the last embedding
            model_output, step_score = self._decode(
                seqs=self.seqs[:, self.step_nr - 1 : self.step_nr],
                padding_mask=None,
            )

            self.state_bag.increment_step_nr()

        # model_output: EmbeddingBag
        return self.finalize_step(model_output, step_score)

    def finalize_step(
        self, model_output: torch.Tensor, step_score: torch.Tensor
    ) -> bool:
        """Post-processing and finalizing a step
        by checking all stopping criteria
        Takes the model's outputed embeddings (model_output)
        and their associated scores (step_score)
        If we're stepping, return True, else return False
        """
        already_finished = self.lengths > -1
        should_finish_now = torch.zeros_like(already_finished)

        model_last_output = model_output[:, -1]
        device = model_last_output.device

        # Ignore prompt positions between min-max prompt_len
        must_keep_going = None
        if self.step_nr < self.max_prompt_len:
            assert self.prompt_padding_mask is not None, (
                f"If self.prompt_padding_mas is None, then self.step_nr should start from self.max_prompt_len={self.max_prompt_len} - currently self.step_nr = {self.step_nr}"
            )
            mask = self.prompt_padding_mask[:, self.step_nr]
            model_last_output[mask] = self.seqs[mask, self.step_nr]
            must_keep_going = mask

        # Check stopping based on EOS similarity.
        if self.eos_threshold is not None and self.eos_vec is not None:
            sim2eos = torch.nn.functional.cosine_similarity(
                self.eos_vec.to(device), model_last_output
            )
            logger.debug(f"Similarity to eos vector: {sim2eos} vs {self.eos_threshold}")
            should_finish_now = should_finish_now | sim2eos.ge(self.eos_threshold)

        # Check stopping based on repetition.
        if (
            self.options.stop_on_repetition_cosine_threshold is not None
            and self.step_nr > 0
        ):
            sim2prev = torch.nn.functional.cosine_similarity(
                self.seqs[:, self.step_nr - 1], model_last_output
            )
            logger.debug(
                f"Similarity to prev vector: {sim2prev} vs {self.options.stop_on_repetition_cosine_threshold}"
            )
            should_finish_now = should_finish_now | sim2prev.ge(
                self.options.stop_on_repetition_cosine_threshold
            )

        if must_keep_going is not None:
            logger.debug(
                f"Must keep going (to cover max_prompt_len={self.max_prompt_len}) is not None = {must_keep_going}"
            )
            should_finish_now = should_finish_now & ~must_keep_going

        # Keep going if output is shorter than min_gen_len:
        if self.prompt_seq_lens is not None:
            longer_than_min_gen_len = (self.step_nr - self.prompt_seq_lens).ge(
                self.min_gen_len
            )
        else:
            longer_than_min_gen_len = (
                self.step_nr - self.max_prompt_len
            ) >= self.min_gen_len

        logger.debug(
            f"Longer than min_gen_len ({self.min_gen_len}) = {longer_than_min_gen_len}"
        )
        should_finish_now = should_finish_now & longer_than_min_gen_len
        stopped_on_eos = should_finish_now

        # Stop hypotheses that reached max_gen_len
        if self.prompt_seq_lens is not None:
            exceeds_max_gen_len = (self.step_nr - self.prompt_seq_lens + 1).ge(
                self.max_gen_len
            )
            logger.debug(
                f"step: {self.step_nr}; max_gen_len: {self.max_gen_len}; promt_lens: {self.prompt_seq_lens}; steps exceeded: {self.max_gen_len + self.prompt_seq_lens}"
            )

        else:
            exceeds_max_gen_len = (
                self.step_nr - self.max_prompt_len + 1
            ) >= self.max_gen_len
            logger.debug(
                f"step: {self.step_nr}; max_gen_len: {self.max_gen_len}; promt_lens: None (unique length: {self.max_prompt_len}); steps exceeded: {self.max_prompt_len + self.max_gen_len}"
            )

        logger.debug(
            f"Stopping criteria: {should_finish_now}; exceeds max len: {exceeds_max_gen_len}; already finished: {already_finished}"
        )

        should_finish_now = should_finish_now | exceeds_max_gen_len

        # Assign lengths to the sequences that have just finished.
        should_finish_now = should_finish_now & ~already_finished
        self.lengths[should_finish_now] = self.step_nr + 1

        # Record the current step.
        self.seqs[:, self.step_nr] = model_last_output.squeeze(1)
        self.step_scores[:, self.step_nr - self.min_prompt_len] = step_score[:, -1]

        #  Save completed hypsptheses
        finished_mask = self.lengths.ne(-1)
        finished_indices = finished_mask.nonzero()

        # Remove finished hypotheses and reorder variables/state_bag if any are left
        if len(finished_indices) > 0:
            for idx in finished_indices:
                self.finish_sequence(int(idx), is_eos=bool(stopped_on_eos[int(idx)]))

        active_mask = ~finished_mask
        active_indices = active_mask.nonzero().squeeze(-1)

        if len(active_indices) == 0:
            return False

        self.reorder_state(active_indices)

        return True

    def finish_sequence(self, idx: int, is_eos: bool = False) -> None:
        seq_len = int(self.lengths[idx].item())

        if self.options.trim_hypotheses and self.lengths[idx].item() > -1 and is_eos:
            seq_len = int(self.lengths[idx].item()) - int(
                not self.options.include_eos_token
            )

        sample_idx = int(self.sample_indices[idx])
        self.hypotheses[sample_idx] = [
            Hypothesis(
                seq=self.seqs[idx, :seq_len],
                score=None,
                step_scores=self.step_scores[idx],  # Trim it as well?
            )
        ]

    def state_bag_reorder(self, new_order: torch.Tensor) -> None:
        if self.state_bag is not None:
            self.state_bag.reorder(new_order)

    def reorder_state(self, new_order: torch.Tensor) -> None:
        self.state_bag_reorder(new_order)

        self.seqs = self.seqs.index_select(dim=0, index=new_order)

        self.sample_indices = self.sample_indices.index_select(dim=0, index=new_order)

        self.step_scores = self.step_scores.index_select(dim=0, index=new_order)

        self.lengths = self.lengths.index_select(dim=0, index=new_order)

        if self.prompt_padding_mask is not None:
            self.prompt_padding_mask = self.prompt_padding_mask.index_select(
                dim=0, index=new_order
            )

        if self.prompt_seq_lens is not None:
            self.prompt_seq_lens = self.prompt_seq_lens.index_select(
                dim=0, index=new_order
            )


RuntimeError: operator torchvision::nms does not exist

In [27]:
%cd /content/large_concept_model/lcm/inference/lcm
!uv run generator.py -h

/content/large_concept_model/lcm/inference/lcm


In [31]:
%cd /content/large_concept_model/scripts
!uv run prepare_wikipedia.py

/content/large_concept_model/scripts
2025-08-22 22:34 INFO 11753:datasets - PyTorch version 2.5.1+cpu available.
2025-08-22 22:34 INFO 11753:datasets - Polars version 1.17.1 available.
ERROR: The function received no value for the required argument: output_dir
Usage: prepare_wikipedia.py OUTPUT_DIR

For detailed information on this command, run:
  prepare_wikipedia.py --help


In [32]:
%cd /content/large_concept_model/scripts
!uv run prepare_wikipedia.py OUTPUT_DIR /content

/content/large_concept_model/scripts
2025-08-22 22:35 INFO 11967:datasets - PyTorch version 2.5.1+cpu available.
2025-08-22 22:35 INFO 11967:datasets - Polars version 1.17.1 available.
2025-08-22 22:35 WARNING 11967:WrappedMapper - Output schema will NOT be validated
  0% 0/1 [00:00<?, ?it/s]/content/large_concept_model/.venv/lib/python3.12/site-packages/submitit/core/plugins.py:24: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/content/large_concept_model/.venv/lib/python3.12/site-packages/pkg_resources/__init__.py:3149: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2025-08-22 22:35 INFO 11967:stopes.launcher - for

In [33]:
%cd /content/large_concept_model/lcm/inference/two_tower_diffusion_lcm
!uv run generator.py --help

/content/large_concept_model/lcm/inference/two_tower_diffusion_lcm


In [34]:
%cd /content/large_concept_model/lcm/inference/two_tower_diffusion_lcm
!uv run generator.py

/content/large_concept_model/lcm/inference/two_tower_diffusion_lcm


In [36]:
%cd /content/large_concept_model
!python -m lcm.train \
    +pretrain=mse \
    ++trainer.output_dir="checkpoints/mse_lcm" \
    ++trainer.experiment_name=training_mse_lcm \

/content/large_concept_model
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fa

In [ ]:


!torchrun --standalone

In [37]:

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3  \
  --model_name meta-llama/Llama-3.1-8B-Instruct \
  --generator_batch_size 16 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 200}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 16 \
  --dataset.soure_text_column prompt \
  --dataset.source_target_column answer \
  --dump_dir output_results

usage: __main__.py [-h] [--cfg CFG] [--columns COLUMNS]
                   [--source_text_column SOURCE_TEXT_COLUMN]
                   [--target_text_column TARGET_TEXT_COLUMN]
                   [--source_prefix_text SOURCE_PREFIX_TEXT]
                   [--source_suffix_text SOURCE_SUFFIX_TEXT]
                   [--target_prefix_text TARGET_PREFIX_TEXT]
                   [--target_suffix_text TARGET_SUFFIX_TEXT]
                   [--source_sequences SOURCE_SEQUENCES]
                   [--target_sequences TARGET_SEQUENCES]
                   [--silent_freeze SILENT_FREEZE] [--file_path FILE_PATH]
                   [--prompt_template PROMPT_TEMPLATE]
__main__.py: error: unrecognized arguments: --soure_text_column prompt --source_target_column answer
E0822 22:43:05.943000 13869 torch/distributed/elastic/multiprocessing/api.py:869] failed (exitcode: 2) local_rank: 0 (pid: 13881) of binary: /content/large_concept_model/.venv/bin/python
Traceback (most recent call last):
  File "/co

In [39]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3  \
  --model_name meta-llama/Llama-3.1-8B-Instruct \
  --generator_batch_size 16 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 200}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 16 \
  --source_text_column prompt \
  --target_text_column answer \
  --dump_dir output_results

/content/large_concept_model/lcm
usage: __main__.py [-h] [--cfg CFG] [--model_name MODEL_NAME]
                   [--revision REVISION] [--use_auth_token USE_AUTH_TOKEN]
                   [--tokenizer_name TOKENIZER_NAME]
                   [--tokenizer_revision TOKENIZER_REVISION]
                   [--generator_batch_size GENERATOR_BATCH_SIZE]
                   [--model_parallel_size MODEL_PARALLEL_SIZE]
                   [--cache_dir CACHE_DIR] [--model_class MODEL_CLASS]
                   [--tokenizer_class TOKENIZER_CLASS]
                   [--repetition_penalty REPETITION_PENALTY]
                   [--encoder_repetition_penalty ENCODER_REPETITION_PENALTY]
                   [--encoder_no_repeat_ngram_size ENCODER_NO_REPEAT_NGRAM_SIZE]
                   [--no_repeat_ngram_size NO_REPEAT_NGRAM_SIZE]
                   [--system_prompt SYSTEM_PROMPT]
__main__.py: error: unrecognized arguments: --source_text_column prompt --target_text_column answer
E0822 22:46:00.485000 14587

In [41]:
%cd /content/large_concept_model/lcm/inference/lcm
!uv run generator.py --prompt "a beautiful landscape with trees and river" --output_dir "./my_images"

/content/large_concept_model/lcm/inference/lcm


In [42]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3 \
  --model_name meta-llama/Llama-3.1-8B-Instruct \
  --generator_batch_size 16 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 200}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 16 \
  --dataset.source_text_column prompt \
  --dataset.target_text_column answer \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 22:48:53,115] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 22:48:53,116] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 22:48:53,125] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
[rank0]: Traceback (most recent call last):
[rank0]:   File "<frozen runpy>", line 198, in _run_module_as_main
[rank0]:   File "<frozen runpy>", line 88, in _run_code
[rank0]:   File "/content/large_concept_model/lcm/evaluation/__main__.py", line 59, in <module>
[rank0]:     local.main(cfg, logger=logger)
[rank0]:   File "/content/large_concept_model/lcm/evaluation/cli/local.py", line 38, in main
[rank0]:     metrics, result_file = run_task(run_config, logger=logger, gang=get_gang())
[rank0]:                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
[rank0]:   File "/content/large_concept_model/lcm/evaluation/run.py", line 190, in run_task
[rank0]:     task

In [43]:
%cd /content/large_concept_model/lcm

# إنشاء المجلد المطلوب
!mkdir -p jsonl_dataset/cnn_dailymail

# تحميل وتجهيز البيانات
!python -m lcm.evaluation.tasks.prepare_cnn_dailymail \
  --output_dir jsonl_dataset/cnn_dailymail \
  --split test

/content/large_concept_model/lcm
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-package

In [5]:
%cd /content

# تثبيت مكتبة datasets
#!pip install datasets

# تحميل البيانات مباشرة من Hugging Face
from datasets import load_dataset
import json
import os

dataset = load_dataset("cnn_dailymail", version="3.0.0", split="test")

# تحديد المسار الكامل للمجلد والملف
output_dir = "/content/large_concept_model/lcm/jsonl_dataset/cnn_dailymail"
output_file = os.path.join(output_dir, "test.jsonl")

# إنشاء المجلد إذا لم يكن موجودًا
os.makedirs(output_dir, exist_ok=True)

# حفظ البيانات بالتنسيق المطلوب
with open(output_file, "w") as f:
    for item in dataset:
        # تنسيق البيانات كما يتوقعها النموذج
        data = {
            "prompt": f"Summarize: {item['article']}",
            "answer": item["highlights"]
        }
        f.write(json.dumps(data) + "\n")

print("تم إنشاء ملف البيانات بنجاح!")

/content


ImportError: cannot import name 'load_dataset' from 'datasets' (/content/large_concept_model/lcm/datasets/__init__.py)

In [ ]:
!pip install datasets

In [7]:
%cd /content/large_concept_model/lcm

# إنشاء مجلد البيانات
!mkdir -p jsonl_dataset/cnn_dailymail

# تحميل بيانات مثال (إذا كان هناك script مخصص)
!python -c "
import json
# بيانات مثال للاختبار
sample_data = [
    {'prompt': 'Summarize: This is a test article about technology.', 'answer': 'Technology test summary.'},
    {'prompt': 'Summarize: Another example for evaluation.', 'answer': 'Evaluation example summary.'}
]

with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\\n')
print('تم إنشاء ملف test.jsonl مع بيانات مثال')


/content/large_concept_model/lcm
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
تم إنشاء ملف test.jsonl مع بيانات مثال


In [5]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor llama3 \
  --model_name meta-llama/Llama-3.2-1B-Instruct \
  --generator_batch_size 2 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 100}' \
  --dataset_dir /content/large_concept_model/lcm/jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 2 \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 23:00:05,643] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 23:00:05,643] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 23:00:05,647] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
config.json: 100% 877/877 [00:00<00:00, 2.90MB/s]
model.safetensors: 100% 2.47G/2.47G [00:36<00:00, 67.6MB/s]
generation_config.json: 100% 189/189 [00:00<00:00, 1.21MB/s]
tokenizer_config.json: 100% 54.5k/54.5k [00:00<00:00, 2.02MB/s]
tokenizer.json: 100% 9.09M/9.09M [00:00<00:00, 11.9MB/s]
special_tokens_map.json: 100% 296/296 [00:00<00:00, 1.84MB/s]
[2025-08-22 23:00:57,127] [rank 0] [INFO] Predictor loaded: HFLlamaPredictor
[2025-08-22 23:00:57,366] [rank 0] [INFO] Using rank=0 among world_size=1 to build self._pipeline
[rank0]: Traceback (most recent call last):
[rank0]:   File "<frozen runpy>", line 198, in _run_module_as_main
[rank0]:   File "<frozen runpy>", line 88, in _

In [3]:
!huggingface-cli login --token c

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `read`


In [6]:
%cd /content/large_concept_model/lcm

# حذف الملف المعطوب
!rm jsonl_dataset/cnn_dailymail/test.jsonl

# إنشاء ملف JSONL صحيح
import json

# بيانات مثال صحيحة
sample_data = [
    {"prompt": "Summarize: This is a test article about technology advancements in 2024.", "answer": "Technology advancements in 2024 include AI and quantum computing."},
    {"prompt": "Summarize: Climate change effects are becoming more visible each year.", "answer": "Climate change impacts are increasing globally with rising temperatures."}
]

# كتابة البيانات بشكل صحيح
with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')

print("تم إنشاء ملف test.jsonl جديد بتنسيق صحيح")

/content/large_concept_model/lcm
تم إنشاء ملف test.jsonl جديد بتنسيق صحيح


In [7]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor huggingface \
  --model_name "gpt2" \
  --generator_batch_size 2 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 50}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 2 \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 23:02:22,446] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 23:02:22,447] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 23:02:22,450] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
config.json: 100% 665/665 [00:00<00:00, 3.41MB/s]
model.safetensors: 100% 548M/548M [00:03<00:00, 166MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 651kB/s]
tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 171kB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 15.1MB/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
[2025-08-22 23:02:27,813] [rank 0] [INFO] Predictor loaded: HuggingfacePredictor
[2025-08-22 23:02:27,855

In [8]:
%cd /content/large_concept_model/lcm

# إصلاح أسماء الأعمدة
import json

# بيانات مع الأسماء الصحيحة
sample_data = [
    {"article": "This is a test article about technology advancements in 2024.", "highlights": "Technology advancements in 2024 include AI and quantum computing."},
    {"article": "Climate change effects are becoming more visible each year.", "highlights": "Climate change impacts are increasing globally with rising temperatures."}
]

# كتابة البيانات بالأسماء الصحيحة
with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')

print("تم إنشاء الملف بأسماء الأعمدة الصحيحة: article و highlights")

/content/large_concept_model/lcm
تم إنشاء الملف بأسماء الأعمدة الصحيحة: article و highlights


In [9]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor huggingface \
  --model_name "gpt2" \
  --generator_batch_size 1 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 50}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 1 \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 23:03:57,066] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 23:03:57,067] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 23:03:57,071] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
[2025-08-22 23:03:58,234] [rank 0] [INFO] Predictor loaded: HuggingfacePredictor
[2025-08-22 23:03:58,272] [rank 0] [INFO] Using rank=0 among world_size=1 to build self._pipeline
[2025-08-22 23:04:05,129] [rank 0] [INFO] Using default tokenizer.
[rank0]: Traceback (most recent call last):
[rank0]:   File "<frozen runpy>", line 198, in _run_module_as_main
[rank0]:   File "<frozen runpy>",

In [10]:
%cd /content/large_concept_model/lcm

!pip install datasets

import json
from datasets import load_dataset

# تحميل بيانات حقيقية
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:5]")  # 5 عينات فقط

# إنشاء الملف بالأسماء الصحيحة
with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in dataset:
        data = {
            "article": item["article"],
            "highlights": item["highlights"]
        }
        f.write(json.dumps(data) + '\n')

print("تم إنشاء ملف ببيانات حقيقية بالأسماء الصحيحة")

/content/large_concept_model/lcm


ImportError: cannot import name 'load_dataset' from 'datasets' (/content/large_concept_model/lcm/datasets/__init__.py)

In [11]:
%cd /content/large_concept_model/lcm

# تثبيت بيانات NLTK المطلوبة
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')

/content/large_concept_model/lcm


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### aشغال

In [12]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor huggingface \
  --model_name "gpt2" \
  --generator_batch_size 1 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 50}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 1 \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 23:05:34,407] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 23:05:34,408] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 23:05:34,413] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
[2025-08-22 23:05:35,606] [rank 0] [INFO] Predictor loaded: HuggingfacePredictor
[2025-08-22 23:05:35,645] [rank 0] [INFO] Using rank=0 among world_size=1 to build self._pipeline
[2025-08-22 23:05:39,743] [rank 0] [INFO] Using default tokenizer.
[2025-08-22 23:05:45,100] [rank 0] [INFO] Using default tokenizer.
[2025-08-22 23:05:45,101] [rank 0] [INFO] Writing raw results to output_resul

In [13]:
%cd /content/large_concept_model/lcm

# عرض نتائج المقاييس
!cat output_results/results/cnn_dailymail_llm.test.json

# عرض الملفات المنشأة
!ls -la output_results/

/content/large_concept_model/lcm
{
    "results": {
        "rouge2": 0.0897674418604651,
        "rougeL": 0.12735042735042734,
        "rougeLsum": 0.12735042735042734,
        "ngram_overlap": 0.6780121922065503,
        "repetition_4": 1.8505525394769475
    },
    "configs": {
        "dataset": {
            "columns": [
                "article",
                "highlights"
            ],
            "source_text_column": "article",
            "target_text_column": "highlights",
            "source_prefix_text": "[INST] Summarize the following article: ",
            "source_suffix_text": " [/INST]",
            "target_prefix_text": null,
            "target_suffix_text": null,
            "source_sequences": null,
            "target_sequences": null,
            "silent_freeze": false,
            "file_path": "jsonl_dataset/cnn_dailymail/test.jsonl",
            "prompt_template": null
        },
        "dataset_dir": "jsonl_dataset/cnn_dailymail",
        "split": "test"

O] Writing metric logs to output_results/metrics.eval.jsonl
🎉 مبروك! لقد نجحت في تشغيل التقييم بنجاح!

النتائج التي حصلت عليها:
لقد عمل التقييم بشكل صحيح وها هي النتائج:

📊 مقاييس الأداء:
Rouge-2: 0.089767 (مقياس للتلخيص)

Rouge-L: 0.127350 (مقياس للتلخيص)

Rouge-LSum: 0.127350 (مقياس للتلخيص)

Ngram Overlap: 0.678012 (تداخل n-gram)

Repetition-4: 1.850553 (معدل التكرار)

📁 الملفات التي تم إنشاؤها:
النتائج الخام: output_results/raw_results/

نتائج المقاييس: output_results/results/cnn_dailymail_llm.test.json

سجلات Tensorboard: output_results/tb/

سجلات المقاييس: output_results/metrics.eval.jsonl

📈 تحليل النتائج:
النتائج تبين أن نموذج GPT-2 قام بـ:

✅ معالجة بيانات CNN/DailyMail بنجاح

✅ إنشاء تلاخيص للarticles

✅ حساب مقاييس التقييم المختلفة



In [ ]:
# مع بيانات أكثر
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor huggingface \
  --model_name "gpt2-medium" \
  --generator_batch_size 1 \
  --tasks cnn_dailymail_llm.test \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 1 \
  --dump_dir output_results_large

In [15]:
!python -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "lcm_gpt2" \
  --device "cpu"

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [16]:
!uv run lcm.convert \
  --model_name "gpt2" \
  --output_dir "lcm_gpt2" \
  --device "cpu"

error: Failed to spawn: `lcm.convert`
  Caused by: No such file or directory (os error 2)


In [17]:
%cd /content/large_concept_model

# تحويل نموذج موجود إلى LCM
python -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "./converted_model" \
  --device "cpu"

SyntaxError: invalid syntax (ipython-input-1559574524.py, line 4)

In [ ]:
from lcm import LCModel

# تحميل النموذج المحول
model = LCModel.from_pretrained("./converted_model")

# استخدام النموذج بالمفاهيم
concepts = model.extract_concepts("Your text here")

In [ ]:
# تقييم النموذج المحول
python -m lcm.evaluation \
  --predictor lcm \
  --model_path "./converted_model" \
  --tasks "concept_understanding_task"

In [ ]:
%cd /content/large_concept_model

# استخدام uv لتشغيل عملية التحويل
uv run python -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model" \
  --concept_dim 256

In [ ]:
# الطريقة المثلى مع uv
!uv run --no-cache \
  -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model" \
  --concept_dim 256 \
  --device "cpu"

In [ ]:
uv run -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model_output" \
  --concept_dim 512 \          # حجم أبعاد المفاهيم
  --batch_size 4 \             # حجم الدفعة
  --device "cuda" \            # استخدام GPU
  --num_workers 2 \            # عدد العاملين
  --max_length 2048 \          # الطول الأقصى
  --save_format "safetensors"  # تنسيق الحفظ

In [ ]:
%cd /content/large_concept_model

# أولاً: التأكد من تثبيت المتطلبات
!uv sync

# ثانياً: تحويل النموذج
uv run -m lcm.convert \
  --model_name "gpt2" \          # بداية بنموذج أصغر
  --output_dir "./gpt2_lcm" \
  --concept_dim 128 \
  --device "cuda" \
  --batch_size 8

In [ ]:
# للنماذج الأصغر (أسرع)
uv run -m lcm.convert \
  --model_name "microsoft/DialoGPT-small" \
  --output_dir "./dialogpt_lcm" \
  --concept_dim 192

# للنماذج المتوسطة
uv run -m lcm.convert \
  --model_name "gpt2-medium" \
  --output_dir "./gpt2_medium_lcm" \
  --concept_dim 256 \
  --device "cuda"

In [ ]:
uv run -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "./lcm_model" \
  --concept_dim 64 \      # تقليل أبعاد المفاهيم
  --batch_size 2 \        # تقليل حجم الدفعة
  --device "cpu"          # استخدام CPU إذا لزم الأمر

In [ ]:
uv run python -c "
from lcm import LCModel
model = LCModel.from_pretrained('./gpt2_lcm')
print('تم تحويل النموذج بنجاح!')
print(f'أبعاد المفاهيم: {model.concept_dim}')
"

In [ ]:
# مع تفعيل logging مفصل
uv run -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "./lcm_model" \
  --concept_dim 256 \
  --log_level "INFO"

In [ ]:
لاستخدام `uv` بدلاً من `python` مباشرة لتحويل النموذج إلى LCM، إليك الطريقة الصحيحة:

## 🔧 الطريقة باستخدام uv:

```bash
%cd /content/large_concept_model

# استخدام uv لتشغيل عملية التحويل
uv run python -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model" \
  --concept_dim 256
```

## 💡 أو بشكل أكثر كفاءة مع uv:

```bash
# الطريقة المثلى مع uv
uv run --no-cache \
  -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model" \
  --concept_dim 256 \
  --device "cuda"
```

## ⚙️ معلمات إضافية مهمة:

```bash
uv run -m lcm.convert \
  --model_name "llama-3-8b" \
  --output_dir "lcm_model_output" \
  --concept_dim 512 \          # حجم أبعاد المفاهيم
  --batch_size 4 \             # حجم الدفعة
  --device "cuda" \            # استخدام GPU
  --num_workers 2 \            # عدد العاملين
  --max_length 2048 \          # الطول الأقصى
  --save_format "safetensors"  # تنسيق الحفظ
```

## 🚀 مثال عملي كامل:

```bash
%cd /content/large_concept_model

# أولاً: التأكد من تثبيت المتطلبات
uv sync

# ثانياً: تحويل النموذج
uv run -m lcm.convert \
  --model_name "gpt2" \          # بداية بنموذج أصغر
  --output_dir "./gpt2_lcm" \
  --concept_dim 128 \
  --device "cuda" \
  --batch_size 8
```

## 📝 لنماذج مختلفة:

```bash
# للنماذج الأصغر (أسرع)
uv run -m lcm.convert \
  --model_name "microsoft/DialoGPT-small" \
  --output_dir "./dialogpt_lcm" \
  --concept_dim 192

# للنماذج المتوسطة
uv run -m lcm.convert \
  --model_name "gpt2-medium" \
  --output_dir "./gpt2_medium_lcm" \
  --concept_dim 256 \
  --device "cuda"
```

## ⚠️ ملاحظات مهمة:

1. **التأكد من توفر الذاكرة**:
```bash
# مراقبة الذاكرة أثناء التشغيل
nvidia-smi
```

2. **إذا كانت الذاكرة غير كافية**:
```bash
uv run -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "./lcm_model" \
  --concept_dim 64 \      # تقليل أبعاد المفاهيم
  --batch_size 2 \        # تقليل حجم الدفعة
  --device "cpu"          # استخدام CPU إذا لزم الأمر
```

3. **التحقق من النموذج المحول**:
```bash
uv run python -c "
from lcm import LCModel
model = LCModel.from_pretrained('./gpt2_lcm')
print('تم تحويل النموذج بنجاح!')
print(f'أبعاد المفاهيم: {model.concept_dim}')
"
```

## 🔍 لمشاهدة التقدم:

```bash
# مع تفعيل logging مفصل
uv run -m lcm.convert \
  --model_name "gpt2" \
  --output_dir "./lcm_model" \
  --concept_dim 256 \
  --log_level "INFO"
```

**جرب هذه الأوامر وسأكون هنا لمساعدتك إذا واجهت أي مشاكل!**

In [24]:
%cd /content/large_concept_model

# أولاً: التأكد من تثبيت المتطلبات
!uv sync

# ثانياً: تحويل النموذج
!uv run -m lcm.convert \
  --model_name "openai-community/gpt2" \          # بداية بنموذج أصغر
  --output_dir "/content/large_concept_model" \
  --concept_dim 128 \
  --device "cpu" \
  --batch_size 8

IndentationError: unexpected indent (ipython-input-2317152500.py, line 8)

In [ ]:
!pip install --upgrade torchvision

import nltk
nltk.download('all')

In [26]:
%cd large_concept_model

[Errno 2] No such file or directory: 'large_concept_model'
/content/large_concept_model


In [27]:
!uv sync --extra cpu --extra eval --extra data

Resolved 192 packages in 104ms
Installed 98 packages in 2.14s
 + accelerate==1.2.0
 + adapters==1.0.1
 + aiohappyeyeballs==2.4.4
 + aiohttp==3.11.10
 + aiosignal==1.3.2
 + annotated-types==0.7.0
 + attrs==24.2.0
 + bert-score==0.3.13
 + blis==0.7.11
 + blobfile==2.1.1
 + cached-property==2.0.1
 + catalogue==2.0.10
 + cloudpathlib==0.20.0
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + confection==0.1.5
 + contourpy==1.3.1
 + cycler==0.12.1
 + cymem==2.0.10
 + datasets==3.2.0
 + dill==0.3.8
 + docopt==0.6.2
 + editdistance==0.8.1
 + en-core-web-sm==3.7.1 (from https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl)
 + fairscale==0.4.13
 + fairseq2==0.3.0rc1
 + fairseq2n==0.3.0rc1+cpu
 + filelock==3.16.1
 + flatbuffers==24.3.25
 + fonttools==4.55.3
 + frozenlist==1.5.0
 + fsspec==2024.9.0
 + ftfy==6.3.1
 + huggingface-hub==0.25.2
 + humanfriendly==10.0
 + importlib-metadata==7.2.1
 + kiwisolver==1.4.7
 + langcodes==3.5.0
 + lan

In [ ]:
!uv pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
!pip install -e ".[data,eval]"

In [2]:
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu

Using Python 3.12.11 environment at: /usr
Resolved 55 packages in 1.01s
Prepared 2 packages in 27.38s
Uninstalled 6 packages in 675ms
Installed 6 packages in 270ms
 - blobfile==3.0.0
 + blobfile==2.1.1
 - fairseq2==0.4.6
 + fairseq2==0.3.0rc1
 - fairseq2n==0.4.6
 + fairseq2n==0.3.0rc1+cpu
 - packaging==24.2
 + packaging==23.2
 - torch==2.6.0
 + torch==2.5.1
 - triton==3.2.0
 + triton==3.1.0


In [3]:
!pip install -e ".[data,eval]"

Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.5 MB/s eta 0:00:00
  Using cached fairseq2-0.4.6-py3-none-any.whl.metadata (1.7 kB)
  Using cached fairseq2n-0.4.6-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blobfile-3.0.0-py3-none-any.whl.metadata (15 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached fairseq2-0.4.6-py3-none-any.whl (514 kB)
Using cached fairseq2n-0.4.6-cp312-cp312-manylinux_2_28_x86_64.whl (3.3 MB)
Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl (766.6 MB)
U

In [2]:
%cd /content/large_concept_model

# أولاً: التأكد من تثبيت المتطلبات
!uv sync

# ثانياً: تحويل النموذج
!uv run -m lcm.convert \
--model_name "openai-community/gpt2" \
--output_dir "/content/large_concept_model" \
--concept_dim 128 \
--device "cpu" \
--batch_size 8

/content/large_concept_model
Resolved 192 packages in 111ms
Audited 89 packages in 0.39ms
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
ModuleNotFoundError: No module named 'fairseq2'


In [1]:
%cd /content/large_concept_model

# تثبيت المتطلبات
!uv sync

# تحويل النموذج
!uv run -m lcm.convert \
  --model_name "openai-community/gpt2" \
  --output_dir "./gpt2_lcm_model" \
  --concept_dim 128 \
  --device "cpu" \
  --batch_size 8 \
  --max_length 512

/content/large_concept_model
Resolved 192 packages in 98ms
Audited 89 packages in 0.37ms
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
ModuleNotFoundError: No module named 'fairseq2'


In [5]:
# التحقق من الملفات المنشأة
!ls -la ./gpt2_lcm_model/

# اختبار تحميل النموذج
!uv run python -c "
from lcm import LCModel
try:
    model = LCModel.from_pretrained('./gpt2_lcm_model')
    print('✅ تم تحويل النموذج بنجاح!')
    print(f'أبعاد المفاهيم: {model.concept_dim}')
except Exception as e:
    print(f'❌ خطأ: {e}')


ls: cannot access './gpt2_lcm_model/': No such file or directory
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


RuntimeError: operator torchvision::nms does not exist

In [2]:
%cd /content/large_concept_model

# تثبيت fairseq2
!pip install fairseq2

# أو باستخدام uv
!uv add fairseq2

/content/large_concept_model
Resolved 193 packages in 2.36s
Prepared 1 package in 501ms
Uninstalled 1 package in 2ms
Installed 26 packages in 322ms
 + blobfile==2.1.1
 + colorama==0.4.6
 + editdistance==0.8.1
 + fairseq2==0.3.0rc1
 + fairseq2n==0.3.0rc1+cpu
 + filelock==3.16.1
 + fsspec==2024.9.0
 + importlib-metadata==7.2.1
 ~ large-concept-model==0.1.0 (from file:///content/large_concept_model)
 + lxml==4.9.4
 + markdown-it-py==3.0.0
 + mdurl==0.1.2
 + mpmath==1.3.0
 + networkx==3.4.2
 + portalocker==3.0.0
 + psutil==5.9.8
 + pycryptodomex==3.21.0
 + rich==13.9.4
 + sacrebleu==2.4.3
 + sympy==1.13.1
 + tbb==2022.0.0
 + tcmlib==1.2.0
 + tiktoken==0.8.0
 + torch==2.5.1+cpu
 + torcheval==0.0.7
 + zipp==3.21.0


In [4]:
%cd /content/large_concept_model
# بعد تثبيت fairseq2، جرب التحويل مرة أخرى
!uv run -m lcm.convert \
--model_name "openai-community/gpt2" \
--output_dir "./gpt2_lcm_model" \
--concept_dim 128 \
--device "cpu" \
--batch_size 8

/content/large_concept_model
/content/large_concept_model/.venv/bin/python3: No module named lcm.convert


In [5]:
# احذف المجلد الحالي إذا كان فيه مشاكل
!rm -rf /content/large_concept_model

# استنساخ من جديد
!git clone https://github.com/facebookresearch/large_concept_model.git
%cd /content/large_concept_model

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'large_concept_model': No such file or directory
[Errno 2] No such file or directory: '/content/large_concept_model'
/content/large_concept_model


In [ ]:
https://github.com/facebookresearch/large_concept_model.git

In [1]:
!git clone https://github.com/facebookresearch/large_concept_model.git


Cloning into 'large_concept_model'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 532 (delta 152), reused 127 (delta 127), pack-reused 234 (from 1)
Receiving objects: 100% (532/532), 800.00 KiB | 6.67 MiB/s, done.
Resolving deltas: 100% (244/244), done.


In [2]:
%cd /content/large_concept_model

/content/large_concept_model


In [ ]:
# التثبيت الصحيح مع جميع التبعيات
!pip install -e .

# أو باستخدام uv
!uv pip install -e .

In [3]:
!uv pip install -e .

Using Python 3.12.11 environment at: /usr
Resolved 126 packages in 393ms
Prepared 1 package in 875ms
Uninstalled 1 package in 4ms
Installed 1 package in 1ms
 ~ large-concept-model==0.1.0 (from file:///content/large_concept_model)


In [6]:
!uv sync --extra cpu --extra eval --extra data

Using CPython 3.12.11 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 192 packages in 95ms
Installed 187 packages in 1.94s
 + absl-py==2.1.0
 + accelerate==1.2.0
 + adapters==1.0.1
 + aiohappyeyeballs==2.4.4
 + aiohttp==3.11.10
 + aiosignal==1.3.2
 + alabaster==1.0.0
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + attrs==24.2.0
 + babel==2.16.0
 + bert-score==0.3.13
 + blis==0.7.11
 + blobfile==2.1.1
 + botok==0.8.12
 + cached-property==2.0.1
 + catalogue==2.0.10
 + certifi==2024.8.30
 + cffi==1.17.1
 + charset-normalizer==3.4.0
 + click==8.1.7
 + cloudpathlib==0.20.0
 + cloudpickle==3.1.0
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + confection==0.1.5
 + contourpy==1.3.1
 + cycler==0.12.1
 + cymem==2.0.10
 + dacite==1.8.1
 + datasets==3.2.0
 + dill==0.3.8
 + docopt==0.6.2
 + docutils==0.21.2
 + editdistance==0.8.1
 + emoji==2.14.0
 + en-core-web-sm==3.7.1 (from https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.

In [7]:
!uv pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
!pip install -e ".[data,eval]"

Using Python 3.12.11 environment at: /usr
Resolved 14 packages in 529ms
Uninstalled 2 packages in 610ms
Installed 2 packages in 250ms
 - torch==2.6.0
 + torch==2.5.1+cpu
 - torchaudio==2.6.0
 + torchaudio==2.5.1+cpu
Using Python 3.12.11 environment at: /usr
Resolved 42 packages in 794ms
Uninstalled 4 packages in 29ms
Installed 4 packages in 51ms
 - blobfile==3.0.0
 + blobfile==2.1.1
 - fairseq2==0.4.6
 + fairseq2==0.3.0rc1
 - fairseq2n==0.4.6
 + fairseq2n==0.3.0rc1+cpu
 - packaging==24.2
 + packaging==23.2
Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.9 MB/s eta 0:00:00
  Using cached fairseq2-0.4.6-py3-none-any.whl.metadata (1.7 kB)
  Using cached fairseq2n-0.4.6-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
 

In [ ]:
# التحقق من أن lcm مثبت بشكل صحيح
!python -c "import lcm; print('✅ LCM مثبت بنجاح'); print(dir(lcm))"

In [ ]:
!python -m lcm.convert \
--model_name "openai-community/gpt2" \
--output_dir "./gpt2_lcm_model" \
--concept_dim 128 \
--device "cpu" \
--batch_size 8

In [1]:
%cd /content/large_concept_model
# بعد تثبيت fairseq2، جرب التحويل مرة أخرى
!uv run -m lcm.convert \
--model_name "openai-community/gpt2" \
--output_dir "./gpt2_lcm_model" \
--concept_dim 128 \
--device "cpu" \
--batch_size 8

/content/large_concept_model
/content/large_concept_model/.venv/bin/python3: No module named lcm.convert


In [2]:
import pandas as pd

dataset_path = "/content/large_concept_model/sample_data/0_a25e918a7789ecfa_0_0.parquet" # dataset path

df = pd.read_parquet(dataset_path) # load dataset in pandas df

df['split'] = 'train' # adding 'split' column to dataset because of the missing split column

df.to_parquet(dataset_path) # convert dataset to parquest again

FileNotFoundError: [Errno 2] No such file or directory: '/content/large_concept_model/sample_data/0_a25e918a7789ecfa_0_0.parquet'

https://github.com/facebookresearch/large_concept_model/issues/19

In [3]:


!torchrun --standalone --nnodes=1 --nproc-per-node=2 -m lcm.evaluation  \
  --predictor base_lcm --sample_latent_variable False \
  --model_card checkpoints/mse_lcm/checkpoints/step_2000/model_card.yaml \
  --launcher standalone \
  --dataset.parquet_path /mnt/large_concept_model/examples/evaluation/parquet_dataset/cnn_dailymail/0_3e1f58ddc7724a53_0_0.parquet \
  --dataset.source_column prompt_sentences_sonar_emb \
  --dataset.source_text_column prompt_sentences \
  --dataset.target_column answer_sentences_sonar_emb \
  --dataset.target_text_column prompt_sentences \
  --tasks lcm_generation \
  --task_args '{"max_gen_len": 200}' \
  --data_loading.batch_size 4  --generator_batch_size 4 \
  --dump_dir /mnt/large_concept_model/output

W0822 23:41:06.852000 28878 torch/distributed/run.py:792] 
W0822 23:41:06.852000 28878 torch/distributed/run.py:792] *****************************************
W0822 23:41:06.852000 28878 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0822 23:41:06.852000 28878 torch/distributed/run.py:792] *****************************************
Traceback (most recent call last):
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 i

In [5]:

import torch
from fairseq2.gang import FakeGang
from lcm.datasets.configs import ParquetDatasetConfig, EvaluationDataLoadingConfig
from lcm.evaluation.utils.data_utils import ParquetTestDataLoader

dataset = ParquetDatasetConfig(
    parquet_path="YOUR Parquet file",
    source_column="prompt_sentences_sonar_emb",
    source_text_column="prompt_sentences",
    target_column="answer_sentences_sonar_emb",
    target_text_column= "answer_sentences"
)

data_loading = EvaluationDataLoadingConfig(batch_size=1, seed=23, min_length_of_sequences=1, nb_epochs=1)

data_loader = ParquetTestDataLoader(
    data_config=data_loading,
    datasets=[dataset],
    gang=FakeGang(device=torch.device("cuda:0")),
    dtype=torch.float32,
)

cnt = 0
for batch in data_loader.iterate_batches():
    cnt += len(batch)

RuntimeError: operator torchvision::nms does not exist

In [6]:
!torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation  \
  --predictor base_lcm \
  --model_card /content/drive/MyDrive/LCM/checkpoints/mse_lcm/checkpoints/step_10000/model_card.yaml \
  --launcher standalone \
  --dataset.parquet_path /content/drive/MyDrive/LCM/eval_data/0_55ac997a0bfaa427_0_0.parquet \
  --dataset.source_column prompt_sentences_sonar_emb \
  --dataset.source_text_column prompt_sentences \
  --dataset.target_column answer_sentences_sonar_emb \
  --dataset.target_text_column prompt_sentences \
  --tasks lcm_generation \
  --task_args '{"max_gen_len": 200}' \
  --data_loading.batch_size 16  --generator_batch_size 16 \
  --dump_dir /content/drive/MyDrive/LCM/output_results_lcm \

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [13]:
%cd /content/large_concept_model/lcm

# تدريب نموذج LCM
!uv run /content/large_concept_model/lcm/train/lcm/trainer.py \
  --model_name "gpt2" \
  --output_dir "./lcm_model" \
  --dataset "/content/large_concept_model/lcm/jsonl_dataset"

/content/large_concept_model/lcm
/content/large_concept_model/lcm/datasets/dataloader.py:107: SyntaxWarning: invalid escape sequence '\s'
  Toeknized sentences (subword-level) in (\sum_i=1^M N_i, max_len)
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:39: SyntaxWarning: invalid escape sequence '\s'
  ('\s\s+', ' '),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:40: SyntaxWarning: invalid escape sequence '\d'
  ('ln\d ', ''),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:41: SyntaxWarning: invalid escape sequence '\g'
  ('([^༅།་ ]) །', '\g<1>་ །'),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py:43: SyntaxWarning: invalid escape sequence '\d'
  ('([^་།\d] )', '\g<1>-'),
/content/large_concept_model/.venv/lib/python3.12/site-packages/botok/tokenizers/sentencetokenizer.py

In [ ]:
%cd /content/large_concept_model

# استخدام نموذج LCM للاستدلال
!python -m lcm.inference.generate \
  --model_path "./path_to_lcm_model" \
  --prompt "Your text here"

In [ ]:
%cd /content/large_concept_model/examples

# تشغيل الأمثلة المتوفرة
!python basic_usage.py

In [8]:
%cd /content/large_concept_model/lcm

import json
import os

# إنشاء المجلد إذا لم يكن موجوداً
os.makedirs('jsonl_dataset/cnn_dailymail', exist_ok=True)

# بيانات مع الأسماء الصحيحة
sample_data = [
    {"article": "This is a test article about technology advancements in 2024.", "highlights": "Technology advancements in 2024 include AI and quantum computing."},
    {"article": "Climate change effects are becoming more visible each year.", "highlights": "Climate change impacts are increasing globally with rising temperatures."}
]

# كتابة البيانات بالأسماء الصحيحة
with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')

print("تم إنشاء الملف بأسماء الأعمدة الصحيحة: article و highlights")

# التحقق من إنشاء الملف
!ls -la jsonl_dataset/cnn_dailymail/

/content/large_concept_model/lcm
تم إنشاء الملف بأسماء الأعمدة الصحيحة: article و highlights
total 12
drwxr-xr-x 2 root root 4096 Aug 22 23:52 .
drwxr-xr-x 3 root root 4096 Aug 22 23:52 ..
-rw-r--r-- 1 root root  325 Aug 22 23:52 test.jsonl


In [ ]:
python -m lcm.train \
    +finetune=two_tower \
    ++trainer.output_dir="checkpoints/finetune_two_tower_lcm" \
    ++trainer.experiment_name=finetune_two_tower_lcm \
    ++trainer.model_config_or_name=my_pretrained_two_tower

In [15]:

!uv run lcm.train \
    +finetune=two_tower \
    ++trainer.output_dir="checkpoints/finetune_two_tower_lcm" \
    ++trainer.experiment_name=finetune_two_tower_lcm \
    ++trainer.model_config_or_name=my_pretrained_two_tower

error: Failed to spawn: `lcm.train`
  Caused by: No such file or directory (os error 2)


In [ ]:
%cd /content/large_concept_model/lcm

import json
import os

# إنشاء المجلد إذا لم يكن موجوداً
os.makedirs('jsonl_dataset/cnn_dailymail', exist_ok=True)

# بيانات مع الأسماء الصحيحة
sample_data = [
    {"article": "This is a test article about technology advancements in 2024.", "highlights": "Technology advancements in 2024 include AI and quantum computing."},
    {"article": "Climate change effects are becoming more visible each year.", "highlights": "Climate change impacts are increasing globally with rising temperatures."}
]

# كتابة البيانات بالأسماء الصحيحة
with open('jsonl_dataset/cnn_dailymail/test.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + '\n')

print("تم إنشاء الملف بأسماء الأعمدة الصحيحة: article و highlights")

# التحقق من إنشاء الملف
!ls -la jsonl_dataset/cnn_dailymail/

### ayhgشغال

In [16]:
%cd /content/large_concept_model/lcm

!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.evaluation \
  --predictor huggingface \
  --model_name "gpt2" \
  --generator_batch_size 1 \
  --tasks cnn_dailymail_llm.test \
  --task_args '{"max_gen_len": 50}' \
  --dataset_dir jsonl_dataset/cnn_dailymail \
  --data_loading.batch_size 1 \
  --dump_dir output_results

/content/large_concept_model/lcm
[2025-08-22 23:59:55,944] [rank 0] [INFO] Selected task execution: ['cnn_dailymail_llm.test']
[2025-08-22 23:59:55,944] [rank 0] [INFO] Running evaluation on task cnn_dailymail_llm.test
[2025-08-22 23:59:55,950] [rank 0] [INFO] Setting 'cpu' as the default device of the process.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
[2025-08-22 23:59:59,067] [rank 0] [INFO] Predictor loaded: HuggingfacePredictor
[2025-08-22 23:59:59,317] [rank 0] [INFO] Using rank=0 among world_size=1 to build self._pipeline
[2025-08-23 00:00:15,201] [rank 0] [INFO] Using default tokenizer.
[2025-08-23 00:00:22,228] [rank 0] [INFO] Using default tokenizer.
[2025-08-23 00:00:22,232] [rank 0] [INFO] Writing raw results to output_resul

In [17]:
%cd /content/large_concept_model/lcm

# عرض نتائج المقاييس
!cat output_results/results/cnn_dailymail_llm.test.json

# عرض الملفات المنشأة
!ls -la output_results/

# عرض النتائج الخام
!ls -la output_results/raw_results/cnn_dailymail_llm.test/

/content/large_concept_model/lcm
{
    "results": {
        "rouge2": 0.0897674418604651,
        "rougeL": 0.12735042735042734,
        "rougeLsum": 0.12735042735042734,
        "ngram_overlap": 0.6780121922065503,
        "repetition_4": 1.8505525394769475
    },
    "configs": {
        "dataset": {
            "columns": [
                "article",
                "highlights"
            ],
            "source_text_column": "article",
            "target_text_column": "highlights",
            "source_prefix_text": "[INST] Summarize the following article: ",
            "source_suffix_text": " [/INST]",
            "target_prefix_text": null,
            "target_suffix_text": null,
            "source_sequences": null,
            "target_sequences": null,
            "silent_freeze": false,
            "file_path": "jsonl_dataset/cnn_dailymail/test.jsonl",
            "prompt_template": null
        },
        "dataset_dir": "jsonl_dataset/cnn_dailymail",
        "split": "test"

In [18]:
%cd /content/large_concept_model

# إنشاء مجلد للبيانات التدريبية
!mkdir -p training_data

# تثبيت المتطلبات الإضافية للتدريب
!pip install -e ".[train]"

/content/large_concept_model
Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for large-concept-model (pyproject.toml) ... done
  Created wheel for large-concept-model: filename=large_concept_model-0.1.0-py3-none-any.whl size=7805 sha256=304f189fb8a0366a55bc9fa68b7208c86b3f70813ed4c3e5ace5e87f55668838
  Stored in directory: /tmp/pip-ephem-wheel-cache-19lg09mg/wheels/e0/87/32/9c89b474bef734af9b36cb13df39a74c4f0c331eb2b1a29025
Successfully built large-concept-model
  Attempting uninstall: large-concept-model
    Found existing installation: large-concept-model 0.1.0
    Uninstalling large-concept-model-0.1.0:
      Successfully uninstalled large-concept-model-0.1.0


In [19]:
%cd /content/large_concept_model

# إنشاء بيانات تدريبية مثال
import json
import os

os.makedirs('training_data/concept_dataset', exist_ok=True)

# بيانات تدريبية للمفاهيم
training_data = [
    {
        "text": "The cat sat on the mat",
        "concepts": ["cat", "sitting", "mat", "domestic", "resting"]
    },
    {
        "text": "Artificial intelligence is transforming healthcare",
        "concepts": ["AI", "healthcare", "technology", "transformation", "innovation"]
    },
    {
        "text": "Climate change causes extreme weather events",
        "concepts": ["climate", "weather", "environment", "extreme", "causes"]
    }
]

# حفظ بيانات التدريب
with open('training_data/concept_dataset/train.jsonl', 'w') as f:
    for item in training_data:
        f.write(json.dumps(item) + '\n')

print("تم إنشاء بيانات التدريب")

/content/large_concept_model
تم إنشاء بيانات التدريب


In [20]:
%cd /content/large_concept_model

# التدريب الأساسي
!python -m lcm.training.train \
  --model_name "gpt2" \
  --train_data_path "training_data/concept_dataset/train.jsonl" \
  --output_dir "trained_lcm_model" \
  --num_train_epochs 3 \
  --per_device_train_batch_size 2 \
  --learning_rate 5e-5 \
  --concept_dim 256

/content/large_concept_model
Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fa

In [ ]:
!python -m lcm.training.train \
  --model_name "gpt2" \
  --train_data_path "training_data/concept_dataset/train.jsonl" \
  --output_dir "trained_lcm_advanced" \
  --num_train_epochs 5 \
  --per_device_train_batch_size 4 \
  --learning_rate 3e-5 \
  --warmup_steps 100 \
  --logging_steps 10 \
  --save_steps 100 \
  --concept_dim 512 \
  --max_seq_length 512 \
  --gradient_accumulation_steps 2

In [ ]:
# إذا لديك بيانات concept-labeled
!python -m lcm.training.train \
  --model_name "gpt2-medium" \
  --train_data_path "path/to/your/concept/data.jsonl" \
  --output_dir "real_trained_lcm" \
  --num_train_epochs 10 \
  --per_device_train_batch_size 8 \
  --learning_rate 2e-5 \
  --concept_dim 768

In [ ]:
# مراقبة تقدم التدريب
!tensorboard --logdir trained_lcm_model/logs

# أو مشاهدة الملفات مباشرة
!ls -la trained_lcm_model/
!cat trained_lcm_model/training_logs.txt

In [ ]:
# بعد انتهاء التدريب
from lcm import LCModel

model = LCModel.from_pretrained("trained_lcm_model")
print("النموذج جاهز للاستخدام!")
print(f"أبعاد المفاهيم: {model.concept_dim}")

# اختبار النموذج
concepts = model.extract_concepts("Technology is changing the world")
print("المفاهيم المستخرجة:", concepts)

In [ ]:
lcm.train

In [28]:
%cd /content/large_concept_model/lcm/train/lcm

!uv run trainer.py \
  --model_name "gpt2" \
  --train_data_path "training_data/concept_dataset/train.jsonl" \
  --output_dir "trained_lcm_advanced" \
  --num_train_epochs 5 \
  --per_device_train_batch_size 4 \
  --learning_rate 3e-5 \
  --warmup_steps 100 \
  --logging_steps 10 \
  --save_steps 100 \
  --concept_dim 512 \
  --max_seq_length 512 \
  --gradient_accumulation_steps 2

/content/large_concept_model/lcm/train/lcm


In [23]:
%cd /content/large_concept_model

# تحميل نموذج LCM جاهز
!uv run python -c "
from lcm import LCModel
model = LCModel.from_pretrained('facebook/lcm-llama-3-8B')  # أو أي نموذج آخر
print('النموذج جاهز!')


/content/large_concept_model
/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

https://huggingface.co/facebook/layerskip-llama3-8B

In [29]:
!torchrun --standalone --nnodes=1 --nproc-per-node=2 \ -m lcm.train launcher=standalone \ +pretrain=mse \ ++trainer.data_loading_config.max_tokens=1000 \ ++trainer.output_dir="checkpoints/mse_lcm" \ +trainer.use_submitit=false

W0823 00:23:34.773000 39180 torch/distributed/run.py:792] 
W0823 00:23:34.773000 39180 torch/distributed/run.py:792] *****************************************
W0823 00:23:34.773000 39180 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0823 00:23:34.773000 39180 torch/distributed/run.py:792] *****************************************
/usr/bin/python3: can't open file '/content/large_concept_model/lcm/train/lcm/ -m': [Errno 2] No such file or directory
/usr/bin/python3: can't open file '/content/large_concept_model/lcm/train/lcm/ -m': [Errno 2] No such file or directory
E0823 00:23:35.045000 39180 torch/distributed/elastic/multiprocessing/api.py:869] failed (exitcode: 2) local_rank: 0 (pid: 39201) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8

### https://chatgpt.com/c/68a90a47-7ce0-8329-903b-8378684a1034

In [30]:
#source .venv/bin/activate
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.train \
  --config-path recipes/train/ \
  ... # بقية الخيارات حسب إعداد الوصفة


Primary config directory not found.
Check that the config directory '/content/large_concept_model/lcm/train/recipes/train' exists and readable

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
E0823 00:28:54.875000 40450 torch/distributed/elastic/multiprocessing/api.py:869] failed (exitcode: 1) local_rank: 0 (pid: 40466) of binary: /content/large_concept_model/.venv/bin/python
Traceback (most recent call last):
  File "/content/large_concept_model/.venv/bin/torchrun", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/content/large_concept_model/.venv/lib/python3.12/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/content/large_concept_model/.venv/lib/python3.12/site-packages/torch/distributed/run.py", line 919, in main
    run(args)
  File "/content/large_concept_model/.venv/lib/python3.12/site-packages/torch/distributed/r

In [31]:
!uv run --extra data prepare_evaluation_data.py prepare_data \
  --dataset_name=cnn_dailymail \
  --output_dir=jsonl_dataset \
  --source_text_column=article \
  --target_text_column=highlights \
  --version=3.0.0


error: Failed to spawn: `prepare_evaluation_data.py`
  Caused by: No such file or directory (os error 2)


In [32]:
!uv run --extra data prepare_evaluation_data.py embed \
  --input_path=jsonl_dataset/cnn_dailymail/test.jsonl \
  --lang=eng_Latn \
  --output_dir=parquet_dataset/cnn_dailymail


error: Failed to spawn: `prepare_evaluation_data.py`
  Caused by: No such file or directory (os error 2)


In [34]:
%cd /content/large_concept_model

!git pull origin main


/content/large_concept_model
From https://github.com/facebookresearch/large_concept_model
 * branch            main       -> FETCH_HEAD
Already up to date.


In [36]:
!uv run --extra data scripts/prepare_wikipedia.py /content/wikidata -- \
--data_dir 20231101.en \
--split "train [0:50]" \
--batch_size 2


2025-08-23 00:35 INFO 41965:datasets - PyTorch version 2.5.1+cpu available.
2025-08-23 00:35 INFO 41965:datasets - Polars version 1.17.1 available.
2025-08-23 00:35 WARNING 41965:WrappedMapper - Output schema will NOT be validated
  0% 0/1 [00:00<?, ?it/s]/content/large_concept_model/.venv/lib/python3.12/site-packages/submitit/core/plugins.py:24: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/content/large_concept_model/.venv/lib/python3.12/site-packages/pkg_resources/__init__.py:3149: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2025-08-23 00:35 INFO 41965:stopes.launcher - for prep_wiki found 0 already cached arr

In [ ]:
# تنصيب المستودع
!git clone https://github.com/facebookresearch/large_concept_model.git
%cd large_concept_model

# إعداد البيئة الافتراضية
!uv sync --extra cpu --extra data --extra eval
!uv pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
!pip install -e ".[data,eval]"


In [37]:
!export USER=colab



In [38]:
!uv run --extra data scripts/prepare_wikipedia.py /content/test_wiki -- \
--data_dir 20231101.en \
--split "train [0:50]" \
--batch_size 2 \
launcher=local


2025-08-23 00:37 INFO 42629:datasets - PyTorch version 2.5.1+cpu available.
2025-08-23 00:37 INFO 42629:datasets - Polars version 1.17.1 available.
2025-08-23 00:37 WARNING 42629:WrappedMapper - Output schema will NOT be validated
  0% 0/1 [00:00<?, ?it/s]/content/large_concept_model/.venv/lib/python3.12/site-packages/submitit/core/plugins.py:24: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/content/large_concept_model/.venv/lib/python3.12/site-packages/pkg_resources/__init__.py:3149: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
2025-08-23 00:37 INFO 42629:stopes.launcher - for prep_wiki found 0 already cached arr

In [39]:
# 1. تثبيت الحزم المطلوبة
!pip install datasets pyarrow nltk

# 2. تحميل عينة صغيرة من ويكيبيديا (الإنجليزية)
from datasets import load_dataset
ds = load_dataset("wikipedia", "20231101.en", split="train[:50]")  # 50 مقال فقط

# 3. تقسيم الجمل باستخدام NLTK
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def split_sentences(example):
    example["sentences"] = sent_tokenize(example["text"])
    return {"sentences": example["sentences"]}

ds = ds.map(split_sentences, batched=False)

# 4. تحويل إلى Parquet
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df = pd.DataFrame({
    "sentences": ds["sentences"]
})
table = pa.Table.from_pandas(df)
pq.write_table(table, "/content/simple_wiki.parquet")

print("تم حفظ البيانات بنجاح في /content/simple_wiki.parquet")


README.md: 0.00B [00:00, ?B/s]

wikipedia.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found wikipedia.py

In [40]:
from datasets import load_dataset
dataset = load_dataset("parquet", data_files="/path/to/file.parquet")
# أو
dataset = load_dataset("csv", data_files="data.csv")


FileNotFoundError: Unable to find '/path/to/file.parquet'

In [41]:
#!pip install datasets pyarrow nltk
from datasets import load_dataset
# إذا كانت متوفرة Parquet
# ds = load_dataset("wikipedia", "20231101.en", split="train[:50]", streaming=False)
# بدلاً من ذلك، استخدم الطريقة اليدوية التالية:

ds = load_dataset("wikipedia", "20231101.en", split="train[:50]")  # فقط المقالات
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def split_sentences(example):
    example["sentences"] = sent_tokenize(example["text"])
    return {"sentences": example["sentences"]}

ds = ds.map(split_sentences, batched=False)

# تحويل إلى CSV:
ds.to_csv("/content/simple_wiki.csv")

# ثم تحميله:
ds2 = load_dataset("csv", data_files="/content/simple_wiki.csv", split="train")


RuntimeError: Dataset scripts are no longer supported, but found wikipedia.py

In [42]:
!pip install datasets==3.6.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [1]:
from datasets import load_dataset

ds = load_dataset("wikipedia", "20231101.en", split="train[:50]")


The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ValueError: BuilderConfig '20231101.en' not found. Available: ['20220301.aa', '20220301.ab', '20220301.ace', '20220301.ady', '20220301.af', '20220301.ak', '20220301.als', '20220301.am', '20220301.an', '20220301.ang', '20220301.ar', '20220301.arc', '20220301.arz', '20220301.as', '20220301.ast', '20220301.atj', '20220301.av', '20220301.ay', '20220301.az', '20220301.azb', '20220301.ba', '20220301.bar', '20220301.bat-smg', '20220301.bcl', '20220301.be', '20220301.be-x-old', '20220301.bg', '20220301.bh', '20220301.bi', '20220301.bjn', '20220301.bm', '20220301.bn', '20220301.bo', '20220301.bpy', '20220301.br', '20220301.bs', '20220301.bug', '20220301.bxr', '20220301.ca', '20220301.cbk-zam', '20220301.cdo', '20220301.ce', '20220301.ceb', '20220301.ch', '20220301.cho', '20220301.chr', '20220301.chy', '20220301.ckb', '20220301.co', '20220301.cr', '20220301.crh', '20220301.cs', '20220301.csb', '20220301.cu', '20220301.cv', '20220301.cy', '20220301.da', '20220301.de', '20220301.din', '20220301.di

In [2]:
from datasets import load_dataset

ds = load_dataset("wikipedia", "20220301.en", split="train[:50]", trust_remote_code=True)


data/20220301.en/train-00000-of-00041.pa(…):   0%|          | 0.00/1.04G [00:00<?, ?B/s]

data/20220301.en/train-00001-of-00041.pa(…):   0%|          | 0.00/705M [00:00<?, ?B/s]

data/20220301.en/train-00002-of-00041.pa(…):   0%|          | 0.00/558M [00:00<?, ?B/s]


KeyboardInterrupt



In [1]:
from datasets import load_dataset

dataset = load_dataset(
    "wikipedia",
    "20220301.en",
    split="train",
    streaming=True
)
small_samples = dataset.take(20)
for example in small_samples:
    print(example["text"][:200])  # يعرض أول 200 حرف فقط


Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds
Autism is a neurodevelopmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior. Parents often notice signs during the first 
Albedo (; ) is the measure of the diffuse reflection of solar radiation out of the total solar radiation and measured on a scale from 0, corresponding to a black body that absorbs all incident radiati
A, or a, is the first letter and the first vowel of the modern English alphabet and the ISO basic Latin alphabet. Its name in English is a (pronounced ), plural aes. It is similar in shape to the Anci
Alabama () is a state in the Southeastern region of the United States, bordered by Tennessee to the north; Georgia to the east; Florida and the Gulf of Mexico to the south; and Mississippi to the 

In [2]:
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.train \
  --config-path recipes/train/finetune \
  +pretrain=mse \
  ++trainer.output_dir="checkpoints/my_base_lcm" \
  ++trainer.data_loading_config.max_tokens=200 \
  ++trainer.data_loading_config.packing=false \
  ++trainer.checkpoint_every_n_steps=50 \
  ++trainer.save_model_every_n_steps=100


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [3]:
!!uv pip install torch==2.5.0+cpu torchvision==0.19.1+cpu --index-url https://download.pytorch.org/whl/cpu


['\x1bUsing Python 3.12.11 environment at: /usr\x1b',
 '\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠙\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠸\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠼\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠴\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠦\x1b \x1btorch==2.5.0+cpu                                                  

In [4]:
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.train \
  --config-path recipes/train/finetune \
  +pretrain=mse \
  ++trainer.output_dir="checkpoints/my_base_lcm" \
  ++trainer.data_loading_config.max_tokens=200 \
  ++trainer.data_loading_config.packing=false \
  ++trainer.checkpoint_every_n_steps=50 \
  ++trainer.save_model_every_n_steps=100


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [5]:
!!uv pip install torch==2.5.0+cpu torchvision==0.19.1+cpu --index-url https://download.pytorch.org/whl/cpu


['\x1bUsing Python 3.12.11 environment at: /usr\x1b',
 '\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠙\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠹\x1b \x1btorchvision==0.19.1+cpu                                                       \x1b',
 '\x1b[2K\x1b⠸\x1b \x1btorchvision==0.19.1+cpu                                                       \x1b',
 '\x1b[2K\x1b⠸\x1b \x1b                                                                              \x1b',
 '\x1b[2K  \x1b×\x1b No solution found when resolving dependencies:',
 '\x1b  ╰─▶ \x1bBec

In [6]:
!uv run torchrun --standalone --nnodes=1 --nproc-per-node=1 -m lcm.train \
  --config-path recipes/train/finetune \
  +pretrain=mse \
  ++trainer.output_dir="checkpoints/my_base_lcm" \
  ++trainer.data_loading_config.max_tokens=200 \
  ++trainer.data_loading_config.packing=false \
  ++trainer.checkpoint_every_n_steps=50 \
  ++trainer.save_model_every_n_steps=100


Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [7]:
!!uv pip install torch==2.5.0+cpu torchvision==0.19.1+cpu --index-url https://download.pytorch.org/whl/cpu


['\x1bUsing Python 3.12.11 environment at: /usr\x1b',
 '\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠙\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠹\x1b \x1btorch==2.5.0+cpu                                                              \x1b',
 '\x1b[2K\x1b⠹\x1b \x1btorchvision==0.19.1+cpu                                                       \x1b',
 '\x1b[2K\x1b⠸\x1b \x1btorchvision==0.19.1+cpu                                                       \x1b',
 '\x1b[2K\x1b⠸\x1b \x1b                                                                              \x1b',
 '\x1b[2K  \x1b×\x1b No solution found when resolving dependencies:',
 '\x1b  ╰─▶ \x1bBec

In [8]:
!!uv pip install --upgrade torchvision==0.20.1+cu124


['\x1bUsing Python 3.12.11 environment at: /usr\x1b',
 '\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠋\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠙\x1b \x1bResolving dependencies...                                                     \x1b',
 '\x1b[2K\x1b⠙\x1b \x1b                                                                              \x1b',
 '\x1b[2K  \x1b×\x1b No solution found when resolving dependencies:',
 '\x1b  ╰─▶ \x1bBecause there is no version of torchvision==0.20.1+cu124 and you require',
 '\x1b      \x1btorchvision==0.20.1+cu124, we can conclude that your requirements are',
 '\x1b      \x1bunsatisfiable.']

In [9]:
!pip install torch==2.5.0+cpu torchvision==0.19.1+cpu --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.5.0+cpu and torchvision==0.19.1+cpu because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.5.0+cpu
    torchvision 0.19.1+cpu depends on torch==2.4.1

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


https://github.com/facebookresearch/large_concept_model/issues/23

In [ ]:
# clone the repo
!git clone https://github.com/facebookresearch/large_concept_model.git

cd large_concept_model

# Install GPU-enabled Fairseq2 compatible with PyTorch 2.5.1
!pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cu121

# Install other dependencies required for the project
!pip install -e ".[data,eval]" -f /content/large_concept_model

# create directory for prep_wiki data
!mkdir -p /content/large_concept_model/sample_data

# run the data preparation script
!python large_concept_model/scripts/prepare_wikipedia.py /content/large_concept_model/sample_data

# create directory for normalizer.pt file
!mkdir -p /content/large_concept_model/normalizer

# create normalizer.pt file
!python /content/large_concept_model/scripts/fit_embedding_normalizer.py --ds pretraining_data_train --save_path "/content/large_concept_model/normalizer/normalizer_train.pt" --max_nb_samples 1000000

# training script for MSE_LCM
!CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nnodes=1 --nproc-per-node=1 -m large_concept_model.lcm.train \
    launcher=standalone +pretrain=mse \
    ++trainer.data_loading_config.max_tokens=512 \
    ++trainer.output_dir="/content/drive/MyDrive/LCM/checkpoints/mse_lcm" \
    +trainer.use_submitit=false

In [ ]:
/root/.cache/huggingface/hub/datasets--wikipedia

In [10]:
!python /content/large_concept_model/scripts/fit_embedding_normalizer.py --ds pretraining_data_train --save_path "/content/large_concept_model/normalizer/normalizer_train.pt" --max_nb_samples 1000000

Traceback (most recent call last):
  File "/content/large_concept_model/scripts/fit_embedding_normalizer.py", line 16, in <module>
    from lcm.datasets.configs import (
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli

In [11]:
!torchrun --standalone --nnodes=1 --nproc-per-node=1 -m large_concept_model.lcm.train \
    launcher=standalone +pretrain=mse \
    ++trainer.data_loading_config.max_tokens=512 \
    ++trainer.output_dir="/content/drive/MyDrive/LCM/checkpoints/mse_lcm" \
    +trainer.use_submitit=false

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 112, in _get_module_details
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/cli/commands/chatbot.p

In [13]:
%cd /content/large_concept_model

!uv pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu
!uv pip install fairseq2==v0.3.0rc1 --pre --extra-index-url  https://fair.pkg.atmeta.com/fairseq2/whl/rc/pt2.5.1/cpu
!pip install -e ".[data,eval]"

/content/large_concept_model
Using Python 3.12.11 environment at: /usr
Audited 3 packages in 218ms
Using Python 3.12.11 environment at: /usr
Audited 1 package in 127ms
Obtaining file:///content/large_concept_model
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.3 MB/s eta 0:00:00
  Using cached fairseq2-0.4.6-py3-none-any.whl.metadata (1.7 kB)
  Using cached fairseq2n-0.4.6-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (1.2 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blobfile-3.0.0-py3-none-any.whl.metadata (15 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take 

In [1]:
%cd /content/large_concept_model
!python /content/large_concept_model/scripts/fit_embedding_normalizer.py --ds pretraining_data_train --save_path "/content/large_concept_model/normalizer/normalizer_train.pt" --max_nb_samples 10

/content/large_concept_model
Traceback (most recent call last):
  File "/content/large_concept_model/scripts/fit_embedding_normalizer.py", line 16, in <module>
    from lcm.datasets.configs import (
  File "/content/large_concept_model/lcm/__init__.py", line 11, in <module>
    from fairseq2 import setup_extensions
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/__init__.py", line 15, in <module>
    from fairseq2.setup import setup_fairseq2 as setup_fairseq2
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/__init__.py", line 12, in <module>
    from fairseq2.setup._cli import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/__init__.py", line 9, in <module>
    from fairseq2.setup._cli._commands import setup_cli as setup_cli
  File "/usr/local/lib/python3.12/dist-packages/fairseq2/setup/_cli/_commands.py", line 11, in <module>
    from fairseq2.cli.commands.chatbot import RunChatbotHandler
  File "/usr/local/lib/python3.